In [1]:
################################################################################                                                                                                                                                              
################################################################################
#
# FILE: export-month-logit.ipynb
#
# BY: Dmitry Sedov 
#
# CREATED: Fri Mar 13 2020
#
# DESC: This code produces tables with CBG-level restaurant choices.
#
# EXEC:
#      
################################################################################
################################################################################

In [2]:
############################### Libraries ######################################

import sqlalchemy as db
import pandas as pd
import os
import numpy as np

################################################################################

In [3]:
############################### Constants ######################################

year = 2018
month = 10
output_folder_path = '/home/user/projects/urban/data/output/spatial-demand/logit' 

################################################################################

In [4]:
######################### PostgreSQL connection ################################

engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname2}')
connection = engine.connect()

################################################################################

In [5]:
########### CBG-to-restaurants dataset creation SQL statements #################

restaurants_table_statement = """
SELECT
    r.sname_place_id,
    r.brands,
    r.naics_code,
    r.categories,
    r.price,
    r.rating,
    r.review_count,
    r.area_m2,
    r.total_minutes_open,
    r.yelp_phone IS NOT NULL AS phone,
    r.state,
    r.zip_code,
    r.cbsa AS r_cbsa,
    r.cbg AS r_cbg,
    r.latitude,
    r.longitude
FROM
    restaurants as r
;
"""

# Create a restaurants table with visits breakdown by home CBG
restaurants_visits_breakdown_table_statement = """
CREATE TEMPORARY TABLE restaurant_visits_long AS (
    WITH restaurant_visits AS (
        SELECT
            r.sname_place_id,
            r.longitude,
            r.latitude,
            v.raw_visit_counts,
            v.raw_visitor_counts,
            v.visitor_home_cbgs
        FROM
            restaurants AS r
        LEFT JOIN
            visits AS v
        ON  
            r.sname_place_id = v.sname_place_id
        AND
            v.year = {year} 
        AND
            v.month = {month}
    )
    SELECT 
        sname_place_id,
        longitude,
        latitude,
        raw_visit_counts,
        raw_visitor_counts,
        (json_each(visitor_home_cbgs)).*
    FROM
        restaurant_visits
);
ALTER TABLE restaurant_visits_long
RENAME COLUMN key TO home_cbg;
ALTER TABLE restaurant_visits_long
RENAME COLUMN value TO visits_from_home_cbg;

CREATE INDEX restaurant_visits_long_sg_idx
ON restaurant_visits_long (sname_place_id);
CREATE INDEX restaurant_visits_long_cbg_idx
ON restaurant_visits_long (home_cbg);
""".format(year = year, month = month)

# CREATE A TABLE with distances between CBG and restaurant:
# shortest polygon-to point distance and centroid-to-point distance
restaurants_visits_breakdown_distances_table_statement = """
CREATE TEMPORARY TABLE restaurant_visits_long_distances AS (
    SELECT
        r.*,
        ST_SetSRID(ST_Point(r.longitude, r.latitude),4326)::geography AS r_location,
        c.wkb_geometry::geography AS cbg_location,
        ST_Centroid(c.wkb_geometry)::geography as cbg_centroid
    FROM 
        restaurant_visits_long AS r
    LEFT JOIN
        cbgs AS c
    ON
        r.home_cbg = c.censusblockgroup
);

ALTER TABLE restaurant_visits_long_distances
ADD COLUMN distance_closest_point NUMERIC,
ADD COLUMN distance_centroid_point NUMERIC;

UPDATE restaurant_visits_long_distances
SET distance_closest_point = ST_Distance(r_location, cbg_location),
    distance_centroid_point = ST_Distance(r_location, cbg_centroid);
"""

restaurants_visits_breakdown_distances_table_export_statement = """
SELECT 
    sname_place_id,
    home_cbg,
    raw_visit_counts,
    raw_visitor_counts,
    visits_from_home_cbg,
    distance_closest_point,
    distance_centroid_point
FROM
    restaurant_visits_long_distances
;
"""

################################################################################

In [6]:
################## CBG-to-restaurants dataset creation #########################

# Get restaurant characteristics
restaurants_characteristics = pd.read_sql(restaurants_table_statement, 
                                          engine)

# Build a table expanding visits-from-cbgs 
result = engine.execute(restaurants_visits_breakdown_table_statement)

# Build a table with restaurant-cbg distances
result = engine.execute(restaurants_visits_breakdown_distances_table_statement)

# Get the expanded visits from CBGs (and corresponding distances) table
restaurants_visits_breakdown = pd.read_sql(restaurants_visits_breakdown_distances_table_export_statement, 
                                           engine)

# Rename column
restaurants_visits_breakdown.rename(columns = {'visits_from_home_cbg': 'visitors_from_home_cbg'}, 
                                    inplace = True)

################################################################################

In [7]:
########################### Close connection ###################################

engine.dispose()

################################################################################

In [8]:
# Check the results
restaurants_visits_breakdown

,sname_place_id,home_cbg,raw_visit_counts,raw_visitor_counts,visitors_from_home_cbg,distance_closest_point,distance_centroid_point
0,sg:0360db6c9a834294871c9ad572e14dba,540379725061,310,257,12,450.297870,4361.701821
1,sg:03a2f6fe886e4ea59127ae7478d250a4,261635417002,330,160,13,34.186659,569.351508
2,sg:0b80d01785d14b3b8daabe3bc6b912cf,482279508014,918,566,12,2900.780800,3474.384898
3,sg:0b80d01785d14b3b8daabe3bc6b912cf,482279508012,918,566,11,1169.340743,2313.470846
4,sg:0b80d01785d14b3b8daabe3bc6b912cf,483359504002,918,566,9,27550.907929,41892.559127
...,...,...,...,...,...,...,...
3186717,sg:fc642b1e0deb40c0a847ba3bb8ea4c82,010830201023,129,95,5,17.798770,1063.264783
3186718,sg:fdf2f2457ac9492ea12aa8e150a9ea07,180973302021,95,72,9,0.000000,810.203254
3186719,sg:01a720816c0e42d495e4b75aaf0b4ed8,481810018011,296,195,6,8818.478084,15131.079953
3186720,sg:00a594a326b340e188de19a4a7eaf675,530630103042,62,41,5,2215.395713,8708.612979


In [9]:
# Check the results
restaurants_visits_breakdown.dtypes

sname_place_id          object
home_cbg                    object
raw_visit_counts             int64
raw_visitor_counts           int64
visitors_from_home_cbg       int64
distance_closest_point     float64
distance_centroid_point    float64
dtype: object

In [10]:
# Check the results
restaurants_characteristics

,sname_place_id,brands,naics_code,categories,price,rating,review_count,area_m2,total_minutes_open,phone,state,zip_code,r_cbsa,r_cbg,latitude,longitude
0,sg:3d7150bd87cb441f9c52ac4a24e6f69e,None,722511,"[{'alias': 'bbq', 'title': 'Barbeque'}]",1.0,4.0,34.0,126.892555,NaN,True,ok,74464,None,400219781004,35.886263,-94.975156
1,sg:1e484aaa553746b1887e60c96296934f,Krispy Krunchy Chicken,722513,None,NaN,NaN,NaN,310.641279,NaN,False,ca,93230,25260,060310011001,36.303961,-119.636296
2,sg:615e0eb2bfa9405889d6bdb6f590d9f2,None,722511,"[{'alias': 'newamerican', 'title': 'American (...",-1.0,4.0,11.0,353.904797,NaN,True,ny,14571,40380,360734013003,43.328487,-78.191166
3,sg:ed001eec28e546bb8a3fdce63b69d751,Little Caesars,722511,None,NaN,NaN,NaN,1236.128049,NaN,False,pr,00680,32420,720970806001,18.199873,-67.137186
4,sg:4a5b60fe17064abd9832a52566c3c3f2,Dairy Queen,722513,None,NaN,NaN,NaN,211.244876,NaN,False,wv,24874,None,541090031002,37.606191,-81.536343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594369,sg:012cc23c8f1f4727acb7121f23e815a2,None,722511,None,NaN,NaN,NaN,1057.010159,NaN,False,ma,02149,14460,250251602001,42.392452,-71.034409
594370,sg:fe867ca0780e4561bb1b627e1878e3b2,None,722511,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",2.0,3.5,30.0,1204.077228,NaN,True,ar,72113,30780,051190043053,34.884628,-92.383712
594371,sg:79096f487e01483481155420b52b9a7c,None,722511,"[{'alias': 'chinese', 'title': 'Chinese'}]",2.0,3.5,32.0,354.550766,NaN,True,il,60473,16980,170318278014,41.601114,-87.605468
594372,sg:c9d8d57e58724feabe69639e9d67cc52,Krispy Krunchy Chicken,722513,None,NaN,NaN,NaN,511.552185,NaN,False,ms,39503,25060,280470032081,30.432251,-89.030705


In [11]:
# Check the results
restaurants_characteristics.dtypes

sname_place_id     object
brands                 object
naics_code              int64
categories             object
price                 float64
rating                float64
review_count          float64
area_m2               float64
total_minutes_open    float64
phone                    bool
state                  object
zip_code               object
r_cbsa                 object
r_cbg                  object
latitude              float64
longitude             float64
dtype: object

In [12]:
# Convert data type for integers with missing values
restaurants_characteristics['price'] = restaurants_characteristics['price'].astype('Int64')
restaurants_characteristics['review_count'] = restaurants_characteristics['review_count'].astype('Int64')

In [13]:
# Merge the datasets
restaurants_visits_breakdown = pd.merge(restaurants_visits_breakdown,
                                        restaurants_characteristics, 
                                        how = 'left',
                                        on = 'sname_place_id', 
                                        validate = 'many_to_one')

In [14]:
restaurants_visits_breakdown

,sname_place_id,home_cbg,raw_visit_counts,raw_visitor_counts,visitors_from_home_cbg,distance_closest_point,distance_centroid_point,brands,naics_code,categories,...,review_count,area_m2,total_minutes_open,phone,state,zip_code,r_cbsa,r_cbg,latitude,longitude
0,sg:0360db6c9a834294871c9ad572e14dba,540379725061,310,257,12,450.297870,4361.701821,None,722511,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",...,80,1818.220491,NaN,True,wv,25414,47900,540379725031,39.299221,-77.842732
1,sg:03a2f6fe886e4ea59127ae7478d250a4,261635417002,330,160,13,34.186659,569.351508,None,722511,"[{'alias': 'restaurants', 'title': 'Restaurant...",...,4,279.038420,NaN,True,mi,48219,19820,261635414003,42.428723,-83.273817
2,sg:0b80d01785d14b3b8daabe3bc6b912cf,482279508014,918,566,12,2900.780800,3474.384898,None,722511,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}]",...,13,294.576740,NaN,True,tx,79720,None,482279505004,32.239935,-101.473059
3,sg:0b80d01785d14b3b8daabe3bc6b912cf,482279508012,918,566,11,1169.340743,2313.470846,None,722511,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}]",...,13,294.576740,NaN,True,tx,79720,None,482279505004,32.239935,-101.473059
4,sg:0b80d01785d14b3b8daabe3bc6b912cf,483359504002,918,566,9,27550.907929,41892.559127,None,722511,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}]",...,13,294.576740,NaN,True,tx,79720,None,482279505004,32.239935,-101.473059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3186717,sg:fc642b1e0deb40c0a847ba3bb8ea4c82,010830201023,129,95,5,17.798770,1063.264783,None,722511,"[{'alias': 'chinese', 'title': 'Chinese'}]",...,6,1240.636321,NaN,True,tn,38449,None,470559208002,34.992166,-86.846271
3186718,sg:fdf2f2457ac9492ea12aa8e150a9ea07,180973302021,95,72,9,0.000000,810.203254,None,722511,"[{'alias': 'tradamerican', 'title': 'American ...",...,4,184.583476,25110.0,True,in,46235,26900,180973302021,39.826789,-85.990070
3186719,sg:01a720816c0e42d495e4b75aaf0b4ed8,481810018011,296,195,6,8818.478084,15131.079953,None,722511,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",...,49,163.697515,NaN,True,tx,75495,43300,481810018033,33.421458,-96.583125
3186720,sg:00a594a326b340e188de19a4a7eaf675,530630103042,62,41,5,2215.395713,8708.612979,None,722511,"[{'alias': 'pizza', 'title': 'Pizza'}]",...,7,1142.332625,NaN,True,wa,99006,44060,530630103011,47.955513,-117.476563


In [15]:
################## Get device and establishments data ##########################

# PostgreSQL connection
engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname2}')
connection = engine.connect()

# Create a table with CBGs' device count and CBSA affiliation
cbg_table_statement = """
SELECT
    CONCAT(c.statefips, c.countyfips, c.tractcode) AS ct,
    c.censusblockgroup AS cbg,
    c.cbsa AS home_cbsa,
    h.number_devices_residing
FROM
    cbgs AS c
LEFT JOIN
    home AS h
ON
    c.censusblockgroup = h.census_block_group 
AND
    h.year = {year} 
AND
    h.month = {month}
;
""".format(year = year, month = month)

cbg_home = pd.read_sql(cbg_table_statement, engine)
# Remove duplicate CBGs
cbg_home = cbg_home.sort_values(['number_devices_residing'],
                                ascending = False).groupby('cbg').head(1)

# Create an establishments table with CBG affiliation and visits
visits_table_statement = """
SELECT 
    sname_place_id
FROM 
    visits
WHERE
    year = {year} AND
    month = {month}
;
""".format(year = year, month = month)
visits_month = pd.read_sql(visits_table_statement, engine)

establishments_table_statement = """
SELECT 
    sname_place_id,
    naics_first2,
    cbg
FROM 
    establishments
;
"""
establishments = pd.read_sql(establishments_table_statement, engine)

# Close the SQL connection-engine
engine.dispose()

################################################################################

In [16]:
############### Aggregate the establishments on the CBG level ##################

# Count open establishement by cbg
est_open_this_month = pd.merge(establishments,
                               visits_month,
                               how = 'inner',
                               validate = 'one_to_one')
cbg_establishments = est_open_this_month.groupby(['cbg',
                                                  'naics_first2']
                                                ).size().reset_index(name = 'counts')
cbg_establishments = cbg_establishments.pivot(index = 'cbg',
                                              columns = 'naics_first2',
                                              values = 'counts')
cbg_establishments.columns = list(map(lambda x: 'naics' + str(x),
                                      cbg_establishments.columns))
cbg_establishments.fillna(0, inplace = True)
cbg_establishments.reset_index(inplace = True)
# Merge to get the CT of the CBG
cbg_establishments = pd.merge(cbg_establishments, 
                              cbg_home[['cbg','ct']], 
                              how = 'left',
                              on = 'cbg', 
                              validate = 'one_to_one')

# Rename columns
cbg_home.rename(columns = {'cbg': 'home_cbg', 'ct': 'home_ct'}, 
                inplace = True)
cbg_establishments.rename(columns = {'cbg': 'r_cbg', 'ct': 'r_ct'}, 
                          inplace = True)

################################################################################

In [17]:
cbg_home

,home_ct,home_cbg,home_cbsa,number_devices_residing
202970,51013101401,510131014011,47900,34069.0
144825,36061003100,360610031001,35620,26437.0
140609,36005009300,360050093006,35620,14784.0
132231,32003006700,320030067001,29820,13154.0
46424,12095017103,120950171032,36740,12915.0
...,...,...,...,...
220546,01015981903,010159819031,11500,NaN
220547,36081038301,360810383010,35620,NaN
220548,28045990000,280459900000,25060,NaN
220549,26001990000,260019900000,None,NaN


In [18]:
cbg_home.dtypes

home_ct                     object
home_cbg                    object
home_cbsa                   object
number_devices_residing    float64
dtype: object

In [19]:
cbg_establishments

,r_cbg,naics11,naics21,naics22,naics23,naics31,naics32,naics33,naics42,naics44,...,naics54,naics55,naics56,naics61,naics62,naics71,naics72,naics81,naics92,r_ct
0,010010201001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,01001020100
1,010010202001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,01001020200
2,010010202002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,3.0,1.0,2.0,2.0,11.0,1.0,01001020200
3,010010203001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,01001020300
4,010010203002,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7.0,...,1.0,0.0,0.0,0.0,3.0,1.0,1.0,4.0,0.0,01001020300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200293,721519510002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72151951000
200294,721537504004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72153750400
200295,721537504005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72153750400
200296,721537505011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72153750501


In [20]:
cbg_establishments.dtypes

r_cbg       object
naics11    float64
naics21    float64
naics22    float64
naics23    float64
naics31    float64
naics32    float64
naics33    float64
naics42    float64
naics44    float64
naics45    float64
naics48    float64
naics49    float64
naics51    float64
naics52    float64
naics53    float64
naics54    float64
naics55    float64
naics56    float64
naics61    float64
naics62    float64
naics71    float64
naics72    float64
naics81    float64
naics92    float64
r_ct        object
dtype: object

In [21]:
# Convert datatypes
cbg_home['number_devices_residing'] = cbg_home['number_devices_residing'].astype('Int64')
naics_col = [col for col in cbg_establishments if col.startswith('naics')]
cbg_establishments[naics_col] = cbg_establishments[naics_col].astype('Int64')

In [22]:
# Merge the datasets
restaurants_visits_breakdown = pd.merge(restaurants_visits_breakdown,
                                        cbg_home, 
                                        how = 'left',
                                        on = 'home_cbg',
                                        validate = 'many_to_one')

restaurants_visits_breakdown = pd.merge(restaurants_visits_breakdown,
                                        cbg_establishments, 
                                        how = 'left',
                                        on = 'r_cbg',
                                        validate = 'many_to_one')

In [29]:
restaurants_visits_breakdown

,sname_place_id,home_cbg,raw_visit_counts,raw_visitor_counts,visitors_from_home_cbg,distance_closest_point,distance_centroid_point,brands,naics_code,categories,...,naics61,naics62,naics71,naics72,naics81,naics92,r_ct,category1,visits_from_home_cbg,outside_good_count
2042852,sg:e1a1221e3a0f43cf9b92adedbaa1bc45,010010201001,612,441,5,2067.538390,3571.699696,Wendy's,722513,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",...,2,1,1,0,4,0,01001020700,hotdogs,6.938776,6517.398870
2279936,sg:4b8ea3994d9d4345aae23d5f9d2c2b82,010010201001,1323,805,8,2295.988576,3811.078454,McDonald's,722513,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",...,2,1,1,0,4,0,01001020700,burgers,13.147826,6517.398870
2521512,sg:aa20dfafc669471c9125bfc64fbb9caf,010010201001,4416,2534,5,5891.585480,7359.427729,Chick-fil-A,722513,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]",...,0,34,1,3,5,0,01001020500,hotdogs,8.713496,6517.398870
2476557,sg:a8101adf3b5547469536a68ff01e4434,010010201001,2145,1428,7,5152.406576,6633.465403,None,722511,"[{'alias': 'mexican', 'title': 'Mexican'}]",...,1,1,1,2,6,0,01001020500,mexican,10.514706,6517.398870
2298165,sg:9bc86afedafd4762a80ad5ba1f1a7e18,010010201001,1048,788,5,2359.904166,3812.526979,Taco Bell,722513,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",...,0,2,0,0,1,0,01001020400,hotdogs,6.649746,6517.398870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525405,sg:6fd99169e8634b2ea5fd0e4de7d2c72c,721537506021,639,367,6,2458.337340,3181.822644,Wendy's,722513,None,...,0,0,0,0,0,0,72153750400,None,10.446866,1283.743826
866733,sg:534b19295a534199ab8534c7d745b054,721537506021,2752,1762,5,23077.145538,24017.370534,Wendy's,722513,None,...,0,0,1,0,0,0,72113073010,None,7.809308,1283.743826
3181619,sg:534b19295a534199ab8534c7d745b054,721537506022,2752,1762,9,20245.099539,22102.035888,Wendy's,722513,None,...,0,0,1,0,0,0,72113073010,None,14.056754,1305.014091
2525402,sg:6fd99169e8634b2ea5fd0e4de7d2c72c,721537506022,639,367,8,2484.079978,5586.130613,Wendy's,722513,None,...,0,0,0,0,0,0,72153750400,None,13.929155,1305.014091


In [24]:
restaurants_visits_breakdown.dtypes

sname_place_id          object
home_cbg                    object
raw_visit_counts             int64
raw_visitor_counts           int64
visitors_from_home_cbg       int64
distance_closest_point     float64
distance_centroid_point    float64
brands                      object
naics_code                   int64
categories                  object
price                        Int64
rating                     float64
review_count                 Int64
area_m2                    float64
total_minutes_open         float64
phone                         bool
state                       object
zip_code                    object
r_cbsa                      object
r_cbg                       object
latitude                   float64
longitude                  float64
home_ct                     object
home_cbsa                   object
number_devices_residing      Int64
naics11                      Int64
naics21                      Int64
naics22                      Int64
naics23                 

In [25]:
# Get first category for each restaurant
restaurants_visits_breakdown['category1'] = restaurants_visits_breakdown['categories'].apply(lambda x: x[0]['alias'] if x else None)

In [26]:
# Sort by home cbg
restaurants_visits_breakdown.sort_values(by = 'home_cbg', inplace = True)

In [27]:
# Construct the within-cbg market shares
restaurants_visits_breakdown['visits_from_home_cbg'] = (restaurants_visits_breakdown['visitors_from_home_cbg'] *
                                                        restaurants_visits_breakdown['raw_visit_counts'] /
                                                        restaurants_visits_breakdown['raw_visitor_counts'])

In [28]:
# Construct the outside-good choice count
restaurants_visits_breakdown['outside_good_count'] = (
    restaurants_visits_breakdown[
        'visits_from_home_cbg'
    ].groupby(restaurants_visits_breakdown['home_cbg']
             ).transform('sum')
)
# Total choices made 31 * restaurants_visits_breakdown['number_devices_residing']:
restaurants_visits_breakdown['outside_good_count'] = (
    31 * restaurants_visits_breakdown['number_devices_residing'] - 
    restaurants_visits_breakdown['outside_good_count']
)

In [33]:
# Add total establishments column
restaurants_visits_breakdown['est_number'] = restaurants_visits_breakdown[naics_col].sum(axis = 1)

In [35]:
################### Export data to process on local computer ###################

# Export path
logit_file_path = os.path.join(output_folder_path,
                               'data_logit.csv')
# Columns to export
columns_to_export = [
    'home_cbg',
    'home_ct',
    'home_cbsa',
    'sname_place_id',
    'visits_from_home_cbg',
    'visitors_from_home_cbg',
    'raw_visit_counts',
    'raw_visitor_counts',
    'number_devices_residing',
    'outside_good_count',
    'distance_closest_point',
    'distance_centroid_point',
    'brands',
    'category1',
    'naics_code',
    'price', 
    'rating',
    'area_m2',
    'total_minutes_open',
    'phone',
    'r_cbg',
    'r_ct',
    'r_cbsa',
    'est_number'
]
columns_to_export = columns_to_export + naics_col

# Perform export
restaurants_visits_breakdown.to_csv(logit_file_path,
                                    columns = columns_to_export,
                                    index = False)

################################################################################

In [34]:
restaurants_visits_breakdown

,sname_place_id,home_cbg,raw_visit_counts,raw_visitor_counts,visitors_from_home_cbg,distance_closest_point,distance_centroid_point,brands,naics_code,categories,...,naics62,naics71,naics72,naics81,naics92,r_ct,category1,visits_from_home_cbg,outside_good_count,est_number
2042852,sg:e1a1221e3a0f43cf9b92adedbaa1bc45,010010201001,612,441,5,2067.538390,3571.699696,Wendy's,722513,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",...,1,1,0,4,0,01001020700,hotdogs,6.938776,6517.398870,17.0
2279936,sg:4b8ea3994d9d4345aae23d5f9d2c2b82,010010201001,1323,805,8,2295.988576,3811.078454,McDonald's,722513,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",...,1,1,0,4,0,01001020700,burgers,13.147826,6517.398870,17.0
2521512,sg:aa20dfafc669471c9125bfc64fbb9caf,010010201001,4416,2534,5,5891.585480,7359.427729,Chick-fil-A,722513,"[{'alias': 'hotdogs', 'title': 'Fast Food'}]",...,34,1,3,5,0,01001020500,hotdogs,8.713496,6517.398870,86.0
2476557,sg:a8101adf3b5547469536a68ff01e4434,010010201001,2145,1428,7,5152.406576,6633.465403,None,722511,"[{'alias': 'mexican', 'title': 'Mexican'}]",...,1,1,2,6,0,01001020500,mexican,10.514706,6517.398870,23.0
2298165,sg:9bc86afedafd4762a80ad5ba1f1a7e18,010010201001,1048,788,5,2359.904166,3812.526979,Taco Bell,722513,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",...,2,0,0,1,0,01001020400,hotdogs,6.649746,6517.398870,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2525405,sg:6fd99169e8634b2ea5fd0e4de7d2c72c,721537506021,639,367,6,2458.337340,3181.822644,Wendy's,722513,None,...,0,0,0,0,0,72153750400,None,10.446866,1283.743826,1.0
866733,sg:534b19295a534199ab8534c7d745b054,721537506021,2752,1762,5,23077.145538,24017.370534,Wendy's,722513,None,...,0,1,0,0,0,72113073010,None,7.809308,1283.743826,3.0
3181619,sg:534b19295a534199ab8534c7d745b054,721537506022,2752,1762,9,20245.099539,22102.035888,Wendy's,722513,None,...,0,1,0,0,0,72113073010,None,14.056754,1305.014091,3.0
2525402,sg:6fd99169e8634b2ea5fd0e4de7d2c72c,721537506022,639,367,8,2484.079978,5586.130613,Wendy's,722513,None,...,0,0,0,0,0,72153750400,None,13.929155,1305.014091,1.0
